# Customer Support Multi-Agent System

Key six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

## Step 01. Installing CrewAI and Tools

To install crewAI, we need to have Python >=3.10 and <=3.13 installed on your system:

In [ ]:
import sys
print(sys.version)

In [ ]:
pip install crewai

In [ ]:
pip install 'crewai[tools]'

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing Libraries, API and LLMs

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
# setting the open ai model name as environment variable
import os

openai_api_key = "Your OPEN AI API KEY"
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Step 03. Creating Agent (with role, focus, cooperation)

### Customer Support Agent 

In [ ]:
customer_support_agent = Agent(
    role="Senior Customer Support Representative",
    goal="Be the most friendly and helpful "
        "customer support representative in your team",
    backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing customer"
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
    allow_delegration=False, # by default, allow_delegration=True
    verbose=True
)

`allow_delegation=False` because we don't want the customer support agent to delegate its work to another agent</li>

### Support Quality Assurance Agent

This agent is responsible for fact checking everything customer support agent says. Ensuring the customer support agent produces best possible response.
It can delegate task to customer agent (if required)

In [ ]:
quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the "
    "best support quality assurance in your team",
    backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    verbose=True
)

In [ ]:
from IPython.display import Image
Image("img/delegation.png")

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Step 04. Forming Task with Tools, Gaurdrails and Agents

- Import CrewAI tools

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

* **SerperDevTool**: Do Google Search and get relevant search results
* **ScrapeWebsiteTool**: Web scraping and data collection
* **WebsiteSearchTool**: semantic searches within website content.

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

#### Different ways to assign Tools

- Agent Level : Agents can use Tool(s) on any of the task they perform
- Task Level : Agents will only use task level tool(s) to perform task

**NOTE**: Task Tools override Agent Tools

### Creating Tasks
- Passing docs_scrape_tool on the Task Level.

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
    agent=customer_support_agent,
    tools=[docs_scrape_tool]
)

* QA agent is not using any tool
* It will only review the work done by the customer support agent

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=quality_assurance_agent,
)

## Step 05. Creating Crew (with Memory)

#### Memory
- Setting `memory=True` when putting the crew together enables all kinds of memory.

In [ ]:
crew = Crew(
    agents=[customer_support_agent, quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=2,
    memory=True
)

#### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)